<a href="https://colab.research.google.com/github/jguhlin/beaker/blob/main/BioBeaker_Initial_Notebook_Playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Note
This isn't the latest trained model. Training is ongoing and still seems to be improving.

In [4]:
!pip install pyracular
!pip install biobeaker

In [34]:
!wget "https://media.githubusercontent.com/media/jguhlin/beaker/main/precomputed/beaker_medium_tripleloss.data-00000-of-00001"
!wget "https://media.githubusercontent.com/media/jguhlin/beaker/main/precomputed/beaker_medium_tripleloss.index"
!wget "https://media.githubusercontent.com/media/jguhlin/beaker/main/precomputed/weights_wide_singlelayer_k21_3Aug2020model_21_dims_32_epochs256.npy"
!wget "https://media.githubusercontent.com/media/jguhlin/beaker/main/example/Arabidopsis_chr1.sfasta"
!wget "https://media.githubusercontent.com/media/jguhlin/beaker/main/example/Arabidopsis_chr1.sfai"
!wget "https://media.githubusercontent.com/media/jguhlin/beaker/main/example/Vvulg.sfasta"
!wget "https://media.githubusercontent.com/media/jguhlin/beaker/main/example/Vvulg.sfai"

--2020-12-02 22:07:27--  https://media.githubusercontent.com/media/jguhlin/beaker/main/precomputed/beaker_medium_tripleloss.data-00000-of-00001
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71412220 (68M) [application/octet-stream]
Saving to: ‘beaker_medium_tripleloss.data-00000-of-00001.3’

beaker_medium_tripl 100%[===================>]  68.10M   209MB/s    in 0.3s    

2020-12-02 22:07:30 (209 MB/s) - ‘beaker_medium_tripleloss.data-00000-of-00001.3’ saved [71412220/71412220]

--2020-12-02 22:07:30--  https://media.githubusercontent.com/media/jguhlin/beaker/main/precomputed/beaker_medium_tripleloss.index
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to media.githubu

In [6]:
import tensorflow as tf
import os
import tensorflow_addons as tfa
import numpy as np
import time
import pyracular
from biobeaker.utils import get_angles, positional_encoding
from biobeaker import BEAKER
from tensorflow.keras.layers import Dense, Embedding, Flatten, Lambda, Subtract, Input, Concatenate, AveragePooling1D, LocallyConnected1D, Conv1D, GaussianNoise, BatchNormalization, Reshape, GlobalAveragePooling1D, Dropout
from tensorflow.keras.models import Model, Sequential
import pandas as pd
import umap
import plotly.express as px

In [7]:
# Hyper parameters
k = 21
window_size = 7  # up to 511
num_layers = 8
embedding_dims = 32
output_dims = 128 # Output dims are also internal dims!
intermediate_dims = 256
num_heads = 8
dropout_rate = 0.15
max_positions = 512
batch_size = 128

In [8]:
transformer = BEAKER(num_layers, embedding_dims, output_dims, num_heads, intermediate_dims, max_positions,
                          dropout=dropout_rate, attention_dropout=dropout_rate, activation=tfa.activations.mish)

magic = Dense(embedding_dims, 
                activation=tf.nn.swish, 
                name="Magic", 
                use_bias=False, 
                trainable=False,
                dtype=tf.float32)

magic.build((window_size+1,k*5))

#Load up the weights
weights = np.load("weights_wide_singlelayer_k21_3Aug2020model_21_dims_32_epochs256.npy", allow_pickle=True)
magic.set_weights([weights[0][0]])

transformer.load_weights("beaker_medium_tripleloss")

cls = np.asarray([[1] * 105])

In [9]:
# The library itself can support FASTA files, but the generators are all designed to work with SFASTA files
# Google Colab is unable to convert the file here... not sure why.
# pyracular.convert_fasta_to_sfasta("Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz", "Arabidopsis_chr1.sfasta")

In [55]:
kmerwindowsgen = pyracular.FastaKmersGenerator(k, "Arabidopsis_chr1.sfasta", window_size, False, False)
first = next(kmerwindowsgen)

In [56]:
# A kmer is stored as a series of True and False (takes up the least amount of memory!) but essentially 1's and 0's
# A nucleotide is stored as a vector of length 5, one-hot encoded with the 1 corresponding to the nucleotide itself
# So "A" is stored as [1 0 0 0 0]
# A Kmer is concatenated nucleotides, so a k=21 will be 105 (k*5)
first[0][0][0:10] # First 2 kmers

[False, False, False, True, False, False, False, False, True, False]

In [57]:
# This is easier seen as numpy encoded as 0 and 1's, which is how tensorflow views it
np.asarray(first[0][0][0:10], dtype=np.int)

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0])

In [58]:
# With a few functions, we can convert back to string representation
def convert_all_kmers(kmers):
    kmers_as_str = list()
    for x in kmers:
        y = "".join(list(map(convert_letter_to_string, np.array_split(x, k))))
        kmers_as_str.append(y)
    return kmers_as_str

def convert_letter_to_string(x):
    y = np.nonzero(x)[0][0]
    if y == 0:
        return "A"
    elif y == 1:
        return "T"
    elif y == 2:
        return "N"
    elif y == 3:
        return "C"
    elif y == 4:
        return "G"

In [59]:
# Bam
convert_all_kmers(first[0])

['CCCTAAACCCTAAACCCTAAA',
 'CCCTAAACCTCTGAATCCTTA',
 'ATCCCTAAATCCCTAAATCTT',
 'TAAATCCTACATCCATGAATC',
 'CCTAAATACCTAATTCCCTAA',
 'ACCCGAAACCGGTTTCTCTGG',
 'TTGAAAATCATTGTGTATATA']

In [60]:
# We also have the FASTA header info
# Here 1 means chromosome 1 (same as the FASTA header for this particular file)
# If this is from the reverse complement strand
# And the positions of each of the kmers
[first[2], first[3], first[1]]

['1',
 False,
 [(0, 20), (21, 41), (42, 62), (63, 83), (84, 104), (105, 125), (126, 146)]]

In [61]:
# For some applications we use a CLS token
# It's encoded as all 1's
cls = np.asarray([[1] * 105])

In [76]:
kmerwindowsgen = pyracular.FastaKmersGenerator(k, "Arabidopsis_chr1.sfasta", window_size, False, False)

# We can run along the windows of the chromosome...
# This takes awhile, especially since we aren't using TPU/GPU here...
ids = list()
species_id = list()
results = list()
for _x in range(10): # Do 100 batches
    batch = list()
    for _i in range(256): # Of 256 windows each (So 100 * 256 * window_size * k nucleotides)
        x = next(kmerwindowsgen)
        kmers = np.concatenate([cls, x[0]])
        batch.append(kmers)
        ids.append(x[1][0][0])
        species_id.append("Arabidopsis")
    batch = np.asarray(batch)
    enc_outputs, _, _ = transformer(magic(batch), False)
    results.append(enc_outputs[:, 0, :])

# Tensorflow Addons will give a warning, but it's safely ignored...

In [77]:
results_concat = np.concatenate(results)
results_concat.shape

(2560, 256)

In [78]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(results_concat)
embedding.shape

(2560, 2)

In [79]:
df = pd.DataFrame(data={"x": embedding[:, 0], "y":embedding[:, 1], "id":ids})

In [80]:
fig = px.scatter(df, x="x", y="y", hover_name="id", color="id")
fig.update_traces(marker=dict(size=4))
fig.show()

# HERE: Id is the start position along the chromosome of the window

In [81]:
kmerwindowsgen = pyracular.FastaKmersGenerator(k, "Vvulg.sfasta", window_size, False, False)

for _x in range(10): # Do 100 batches
    batch = list()
    for _i in range(256): # Of 256 windows each (So 100 * 256 * window_size * k nucleotides)
        x = next(kmerwindowsgen)
        kmers = np.concatenate([cls, x[0]])
        batch.append(kmers)
        ids.append(x[1][0][0])
        species_id.append("Vespula")
    batch = np.asarray(batch)
    enc_outputs, _, _ = transformer(magic(batch), False)
    results.append(enc_outputs[:, 0, :])

# Tensorflow Addons will give a warning, but it's safely ignored...

In [85]:
[len(species_id), embedding.shape]

[5120, (5120, 2)]

In [83]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(results_concat)

results_concat = np.concatenate(results)
results_concat.shape
df = pd.DataFrame(data={"x": embedding[:, 0], "y":embedding[:, 1], "id":species_id})

In [86]:
fig = px.scatter(df, x="x", y="y", hover_name="id", color="id")
fig.update_traces(marker=dict(size=4))
fig.show()

# HERE id is the species we pulled from. You can see an easy separation

In [87]:
# What if we increase the window size?
window_size = 32

kmerwindowsgen = pyracular.FastaKmersGenerator(k, "Arabidopsis_chr1.sfasta", window_size, False, False)

# We can run along the windows of the chromosome...
# This takes awhile, especially since we aren't using TPU/GPU here...
ids = list()
species_id = list()
results = list()
for _x in range(10): # Do 100 batches
    batch = list()
    for _i in range(256): # Of 256 windows each (So 100 * 256 * window_size * k nucleotides)
        x = next(kmerwindowsgen)
        kmers = np.concatenate([cls, x[0]])
        batch.append(kmers)
        ids.append(x[1][0][0])
        species_id.append("Arabidopsis")
    batch = np.asarray(batch)
    enc_outputs, _, _ = transformer(magic(batch), False)
    results.append(enc_outputs[:, 0, :])

# Tensorflow Addons will give a warning, but it's safely ignored...

kmerwindowsgen = pyracular.FastaKmersGenerator(k, "Vvulg.sfasta", window_size, False, False)

for _x in range(10): # Do 100 batches
    batch = list()
    for _i in range(256): # Of 256 windows each (So 100 * 256 * window_size * k nucleotides)
        x = next(kmerwindowsgen)
        kmers = np.concatenate([cls, x[0]])
        batch.append(kmers)
        ids.append(x[1][0][0])
        species_id.append("Vespula")
    batch = np.asarray(batch)
    enc_outputs, _, _ = transformer(magic(batch), False)
    results.append(enc_outputs[:, 0, :])

# Tensorflow Addons will give a warning, but it's safely ignored...

reducer = umap.UMAP()
embedding = reducer.fit_transform(results_concat)

results_concat = np.concatenate(results)
results_concat.shape
df = pd.DataFrame(data={"x": embedding[:, 0], "y":embedding[:, 1], "id":species_id})

fig = px.scatter(df, x="x", y="y", hover_name="id", color="id")
fig.update_traces(marker=dict(size=4))
fig.show()

# HERE id is the species we pulled from. You can see an easy separation. Window size of 32 instead of 7

In [ ]:
# No real change